In [1]:

import gym
import torch
import matplotlib.pyplot as plt
import numpy as np
from ppo import PPO
from online_collect import online_collect
from rl_utils import moving_average

In [2]:
 actor_lr = 1e-4
critic_lr = 1e-4#面对复杂任务时，这个值最好降低，调到过3e-4，但是结果不是很理想
hidden_dim = 128
gamma = 0.99#平衡过程奖励和结果奖励
lmbda = 0.95
epochs = 3#原来是10
eps = 0.2 
minibatch_size=256#面对复杂任务时，这个值最好变大，详情可见ppo原文，update耗时很严重
device = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")
fixed_epi=10#改的是这个
env_name = 'Pong-ram-v0'
env = gym.make(env_name)
env.seed(0)
torch.manual_seed(0)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
agent = PPO(state_dim, hidden_dim, action_dim, actor_lr, critic_lr, lmbda,
            epochs, eps, gamma, minibatch_size,device)
return_list = []

In [3]:
def online_collect(agent,env,fixed_epi,return_list,gamma):
    transition_dict = {'states': [], 'actions': [], 'next_states': [], 'rewards': [], 'dones': [],'returns':[]}
    total_epi=0
    episode_return = []
    state = env.reset()
    done = False
    while total_epi<fixed_epi:
        state=state.astype(np.float32)
        state/=255
        action = agent.take_action(state)
        next_state, reward, done, _ = env.step(action)
        transition_dict['states'].append(state)
        transition_dict['actions'].append(action)
        transition_dict['next_states'].append(next_state)
        transition_dict['rewards'].append(reward)
        transition_dict['dones'].append(done)
        state = next_state
        episode_return.append(reward)
        if done:
            total_epi+=1
            return_list.append(sum(episode_return))
            returns=compute_returns(episode_return,gamma)
            transition_dict['returns'].extend(returns)
            episode_return = []
            state = env.reset()
            done = False
    return transition_dict
    

In [4]:
for i in range(2000):
    transition_dict=online_collect(agent,env,fixed_epi,return_list,gamma)
    agent.offline_train(transition_dict)
    print(f"iteration{i}: average return of last 100 episodes is {np.mean(return_list[-10:])}")

/Users/didi/Desktop/demo_ppo/monte carlo/v1_0/ppo.py:45: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  state = torch.tensor([state], dtype=torch.float).to(self.device)


NameError: name 'compute_returns' is not defined

In [ ]:
agent.save()

episodes_list = list(range(len(return_list)))
plt.plot(episodes_list, return_list)
plt.xlabel('Episodes')
plt.ylabel('Returns')
plt.title('PPO on {}'.format(env_name))
plt.show()

mv_return = moving_average(return_list, 9)
plt.plot(episodes_list, mv_return)
plt.xlabel('Episodes')
plt.ylabel('Returns')
plt.title('PPO on {}'.format(env_name))
plt.show()

In [ ]:
 # 渲染模型的表现
import time
num_episodes = 1  # 渲染的轮次
for episode in range(num_episodes):
    state = env.reset()
    done = False
    while not done:
        env.render()  # 渲染当前环境
        time.sleep(0.05)
        state=state.astype(np.float32)
        state/=255
        action = agent.take_action(state)  # 使用训练好的代理选择动作
        state, reward, done, _ = env.step(action)  # 执行动作
        print(reward)
    print(f"Episode {episode + 1} finished.")

env.close()  # 关闭环境